As scientists, sometimes our data is 1-dimensional, measurements of a single variable as a function of some other variable, like our lymphocyte counts in part III. But often, we have complex, multi-dimensional data like  classifications of "cancer" or "no cancer" as a function of many variables.

**Images** are frequently an important part of scientific research as well as a good introduction to multi-dimensional data. There are many things you might want to do with an image. Maybe you are looking at bacteria under a microscope and want to automate the process of counting individual cells. Maybe you are looking at a cancer patient and want to locate and classify a tumor. Maybe you want to measure the image quality achievable with some camera. There are many complex things you could do for these tasks. If you expect to work with more biomedical imaging data, I recommend [this tutorial](https://sakibreza.github.io/biomedical/).

Images are just 2D arrays. A stack of images, like a multi-row CT scan, is a 3D array. If you deal with medical images, you might need to read DICOM files, for which I recommend checking out the [pydicom library](https://pydicom.github.io/). For our purposes, we will start with images read from numpy files. 

In part IV, we'll introduce some basic tasks for working with image data.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio

# Reading images

There are two main types of images you will encounter: **grayscale** (a single 2D image) and **RGB** (a 3D array of the 2D images corresponding to the red, green, and blue color channels). 

**Grayscale images**

In medical imaging, we usually work with grayscale. These might be saved as text files, numpy files, or something else. We'll work with a numpy file example. This requires you know the target image shape, so it is good to try to include this in the filename.
```
img_gray = np.fromfile(filename, datatype)
img_gray = img_gray.reshape([num_rows, num_columns])
```

**RGB images**

Your everyday filetypes like .jpg and .png will have the three color channels. We can read them into a 3D array using `imageio.v2.imread`, then isolate each channel into its own 2D array using slicing. Most of these formats already have the image shape stored, so you don't need to reshape.
Apparently there is a `v2` submodule of `imageio` that you should use to avoid getting a warning message.
```
img_rgb = imageio.v2.imread(filename)
reds =   img_rgb[:,:,0]
blues =  img_rgb[:,:,1]
greens = img_rgb[:,:,2]
```

### <mark>EX 4.1 - Opening images with NumPy and imageio</mark>
*Image from ["AI Accurately Detects Lung Cancer in Scans", The Scientist, 2019](https://www.the-scientist.com/news-opinion/ai-accurately-detects-lung-cancer-in-scans-65914)*

*Using the block below, read in the image "lung_1248x1783.npy" (or ".jpg") in grayscale and RGB. Based on the name, the shape should be [1248, 1783]. Try printing out the images.*

In [ ]:
# grayscale
img_gray = np.fromfile('data/lung_1248x1783.npy', dtype=float)
img_gray = img_gray.reshape([1248, 1783])

# rgb
img_rgb = imageio.v2.imread('data/lung_1248x1783.jpg')
reds =   img_rgb[:,:,0]
blues =  img_rgb[:,:,1]
greens = img_rgb[:,:,2]

# Showing images

Rather than use prints, we can use matplotlib to show an image or heatmap of a 2D array using the function `plt.imshow(array_2D)`. This has some optional arguments:
- `cmap` - the color mapping to use, which gives the color to show corresponding to each number value in the array. Default is "viridis". I recommend using "gray" (grayscale). See more [here](https://matplotlib.org/stable/tutorials/colors/colormaps.html).
- `vmin` - the minimum intensity value to show. All numbers at and below this value will be the same, lowest cmap color (black in grayscale).
- `vmax` - the maximum intensity value to show. All numbers at and above this value will be the same, highest cmap color (white in grayscale).

When I plot images, I also like to turn off the axis ticks using `ax.axis('off')`.

In [ ]:
# RGB
fig, ax = plt.subplots(1,3,figsize=[9,2], dpi=300)
# We'll denote each channel with a different colormap.
# I added a negative sign before each image to invert the colormap.
ax[0].imshow(-reds, cmap='Reds')
ax[1].imshow(-greens, cmap='Greens')
ax[2].imshow(-blues, cmap='Blues')
for i in range(3):
    ax[i].axis('off')
fig.suptitle('RGB')
plt.show()

# grayscale
fig, ax = plt.subplots(1,1,figsize=[2.5,2], dpi=300)
ax.imshow(img_gray, cmap='gray')
ax.axis('off')
ax.set_title('grayscale',fontsize=6)
plt.show()


<mark>*Now that you see the image, does anything look suspicious?*</mark>

We'll move forward with our grayscale image, but now you know how to handle RGB data.

# Cropping

You can crop images using 2D slicing, just like we learned before. You can plot your image with a grid to get a better look at where each index falls.

In [ ]:
plt.imshow(img_gray, cmap='gray')
plt.grid()
plt.show()

### <mark>EX 4.2 - Crop an image</mark>

*Create a new array containing a cropped image around the suspicious lesion. Minimize the amount of surrounding tissue (brighter intensities) shown without cropping out any of the lesion. Plot your result.*

## Intensity histograms and windowing

A histogram is a way to approximately visualize the distribution of numerical data. If you have not yet encountered them, Wikipedia has a nice overview [here](https://en.wikipedia.org/wiki/Histogram). This is useful for image analysis 

We can plot a histogram using the matplotlib `hist(array_1d)` function. We must pass in a 1D array, so to get an image histogram, we need to flatten it first. There is an optional argument `bins` where we can specify a number of bins or the values of the bins.

In [ ]:
num_bins = 100
fig, ax = plt.subplots(1,1,dpi=300, figsize=[6,3])
ax.hist(img_gray.flatten(), bins=num_bins, color='blue')
ax.set_ylabel('counts')
ax.set_xlabel('pixel intensities')
ax.set_title('Histogram of pixel intensities')
plt.show()

Play around with `num_bins` to see different distributions. 

We can use this intensity histogram to inform our choices for `vmin` and `vmax` as described above. These are related to the intensity **window width (WW)** and **window level (WL)** by the following relations:
$$
WW = \text{vmax} - \text{vmin}
$$
$$
WL = \frac{\text{vmax} + \text{vmin}}{2}
$$

or equivalently:
$$
\text{vmax} = WL + WW/2
$$
$$
\text{vmin} = WL - WW/2
$$

Changing these values is known as **windowing**. You are moving the "window" to view different intensity values. Even though matplotlib uses vmin and vmax, in medical imaging, radiologists will speak in terms of window width and level, so we adopt this terminology here.

### <mark>EX 4.3 - Window an image</mark>

*Below is a function for visualizing the effects of different windows and levels in histogram and image space. The following block has a "for" loop demonstrating some outputs for various window width/level arguments.
1. Run the two blocks of code and observe the outputs.
2. Try out more values of WW/WL. Hold one variable constant so you can observe the effect of WW and WL alone.
3. What visually changes with WW? With WL?

In [ ]:
def window_img(img, WW, WL, nbins):
    '''
    a function to plot an image and corresponding histogram 
    at the given window width (WW) and window level (WL)
    '''
    # convert WL, WW
    vmax = WL + WW/2
    vmin = WL - WW/2
    
    # initialize the subplots
    fig, ax = plt.subplots(1, 2, figsize=[8,3.5], dpi=300, width_ratios=[1,2])

    # plot histogram first
    data = img.flatten()
    data_windowed = data[np.where((data > vmin) & (data < vmax))]
    bins = np.linspace(np.min(data), np.max(data), nbins)  # use the same bins for both histograms

    ax[0].hist(data, bins=bins, color='gray', alpha=0.5, label='all intensities')
    ax[0].hist(data_windowed, bins=bins, color='r', alpha=0.7, label='displayed intensities')
    ax[0].legend()
    ax[0].set_title('Pixel values')
    ax[0].set_yticks([]) # remove y axis

    # plot image
    m = ax[1].imshow(img, vmin=vmin, vmax=vmax, cmap='gray')
    ax[1].axis('off')
    ax[1].set_title(f'Windowed image (WL/WW = {WL}/{WW})')
    fig.colorbar(m, ax=ax[1])

    fig.tight_layout()
    plt.show()

    return None

In [ ]:
# set the number of histogram bins
nbins=100

# iterate over window levels and window widths
for WW, WL in [[300, 125]
                [50, 125]
                [50, 40]
                [50, 210]]:
    window_img(img_gray, WL, WW, nbins)

Each image's new appearance corresponds to a sort of "window" moving along the histogram intensities, hence the name "windowing".

# Masking

You can start using conditions to manipulate images. We can create a **mask**, a new image the same shape as our old image, where each pixel value corresponds to the boolean result of some condition. In medical imaging, you may hear about "segmenting" different regions of an image. For example, you might want to only look at a suspicious mass, or the lungs, or some other body region. This is done with masking.

Numpy has a nice module `ma` for dealing with masks. We can use the `np.ma.masked_where` function to create a mask_array object for one or more conditions:
```
mask_array = np.ma.masked_where(condition, img)          # single condition
mask_array = np.ma.masked_where((cond1) & (cond2), img)  # two or more conditions
```
Each `condition` should be some boolean dealing with `img`, like `img > 101.5` or `img != 55`. Even though `img` is an array, this comparison is element-wise.

The output object `mask_array` has two main attributes we want to understand:
- `data` - the original array
- `mask` - the mask

See below for an example.

In [ ]:
# create a mask of values above 100 and below 200
mask_array = np.ma.masked_where((img_gray > 100) & (img_gray < 200), img_gray)
data = mask_array.data
mask = mask_array.mask

# show the data and mask attributes
fig, ax = plt.subplots(1, 2, figsize=[8,3], dpi=300)
ax[0].imshow(data, cmap='gray')
ax[0].set_title('Data')
ax[1].imshow(mask, cmap='gray')
ax[1].set_title('Mask')
for axi in ax:
    axi.axis('off')
fig.tight_layout()
plt.show()

**Manually create a mask**.
You can also create your own mask by initializing a numpy array of the same shape as your image with `dtype=bool`. 0 is False, 1 is True. You can use indexing and slicing to change the values to what you want.

**Extract mask values**.
You can apply the mask to your data to extract the data values ONLY corresponding to the locations where the mask is True. The syntax looks like:
```
mask_values = data[mask]
```
where `data` and `mask` must be the same shape, but `mask_values` will be a 1D array.

This can be useful if you want to compute the average or standard deviation values in a specific area.

### <mark>EX 4.4 - Analyze an image </mark>
*Using your cropped image from before:*
1. Create a mask of the lesion
2. Compute the mean and standard deviation intensity in the lesion

## Drawing ROIs

Masks are nice for dealing with pixels that correspond to a specific condition, letting us handle unusual shapes in the image. But sometimes a plain old rectangle will do. We can also extract **regions of interest (ROIs)** from our images for this purpose. 

The ROIs themselves are just cropped images, like the one you created earlier. For scientific figures, it is often useful to visualize this on an image. Matplotlib has a nice module `patches` we can use for this purpose. (Note the absence of dot pyplot here!)

In [ ]:
from matplotlib import patches

In [ ]:
x0, y0 = 500, 200   # coordinates of upper left corner of ROI
dx, dy = 100, 500   # width (x), height (y)

# initialize plot
fig, ax = plt.subplots(1,1)
ax.imshow(img_gray, cmap='gray')

# create and add a rectangular patch as the ROI
rect = patches.Rectangle((x0, y0), dx, dy, linewidth=1, edgecolor='red', facecolor='none')
ax.add_patch(rect)

# show
plt.show()

### <mark>EX 4.5 - Use ROIs to measure image quality </mark>
*The contrast-to-noise ratio (CNR) is a metric for assessing image quality. It tells you how well a signal, like a suspicious lesion, stands out from the background in an image. It also accounts for noise, or speckling (imagine a staticky TV screen), in an image. If there is a lot of noise, it doesn't matter how much contrast you have, because it is lost to the speckling. A higher CNR means better images.*

1. Draw an ROI **inside** the lesion volume. This is your "signal".
2. Draw a second ROI **outside** the lesion in the surrounding lung. This is the "background".
3. Compute the mean ($\mu$) and standard deviation ($\sigma$) in each ROI.
4. Calculate the contrast-to-noise ratio (CNR):
$$
CNR = \frac{\mu_{\text{signal}} - \mu_{\text{background}}}
{\sqrt{\sigma_{\text{signal}}^2 + \sigma_{\text{background}}^2}}
$$

